In [1]:
import stanza
import pandas as pd
from typing import List, Tuple

In [4]:
stanza.download('uk')

2020-05-09 21:39:20 INFO: Downloading default packages for language: uk (Ukrainian)...
2020-05-09 21:52:54 INFO: Finished downloading models and saved to /home/dbabenko/stanza_resources.


In [2]:
nlp = stanza.Pipeline('uk')

2020-05-13 16:55:05 INFO: Loading these models for language: uk (Ukrainian):
| Processor | Package |
-----------------------
| tokenize  | iu      |
| mwt       | iu      |
| pos       | iu      |
| lemma     | iu      |
| depparse  | iu      |

2020-05-13 16:55:05 INFO: Use device: gpu
2020-05-13 16:55:05 INFO: Loading: tokenize
2020-05-13 16:55:07 INFO: Loading: mwt
2020-05-13 16:55:07 INFO: Loading: pos
2020-05-13 16:55:08 INFO: Loading: lemma
2020-05-13 16:55:08 INFO: Loading: depparse
2020-05-13 16:55:09 INFO: Done loading processors!


In [278]:
train_df = pd.read_csv("/home/dbabenko/DS/NLP/Detect-emotion-sentimental/dataset/booking/booking-train.csv")

In [279]:
train_df.head()

,title,pos_text,neg_text,ratingValue,bestRating,hotel,rating
0,Лише дівчата на рецепції - три рази мені мінял...,Лише дівчата на рецепції - три рази мені мінял...,"Все. Одного досвіду вистарчило, щоб більше сюд...",4.6,10.0,verhovina.uk.html,2
1,"Оформлення кімнати хороше, досить приємне, на ...","Оформлення кімнати хороше, досить приємне, на ...",Nan,9.2,10.0,verhovina.uk.html,5
2,"Усе відмінно, завдяки якісному сервісу ми завж...","Усе відмінно, завдяки якісному сервісу ми завж...",Рекомендую людям котрі подорожують власним тра...,10.0,10.0,verhovina.uk.html,5
3,Ціна/якість в принципі,Ціна/якість в принципі,"душова, трохи старий ремонт, але за 500 грн за...",7.5,10.0,verhovina.uk.html,4
4,"Приїхали з сином біля сьомої ранку, сонні та в...","Приїхали з сином біля сьомої ранку, сонні та в...",Nan,8.3,10.0,verhovina.uk.html,4


### Ananlyze collocations with noun

In [128]:
class TokenNode:
    def __init__(self, token_info, head=None):
        self.token_info = token_info
        self.head = head
        self.children = []
        
    def get_pos(self):
        return self.token_info['upos']
        
    def get_deprel(self):
        return self.token_info['deprel']
    
    def get_head_id(self):
        return self.token_info['head']
    
    def get_id(self):
        return int(self.token_info['id'])
    
    def get_text(self):
        return self.token_info['text']
    
    def get_feats(self):
        return self.token_info['feats']

In [353]:
class NounCollocationExtractor:
    SUPPORTED_NOUN_CHILD_DEPREL = ['amod', 'advmod', 'nmod']
    IGNORE_NOUN_CHILD_DEPREL = ['nummod:gov']
    SUPPORTED_NOUN_HEAD_DEPREL = ['nsubj']

    SUPPORTED_HEAD_POS = ['ADJ', 'VERB']

    def __init__(self, nlp):
        self.nlp = nlp

    def extract(self, text: str):
        doc = self.nlp(text)

        result = []
        for sent in doc.sentences:
            result += self.__extract_from_sent(sent.to_dict())

        return result

    def __extract_from_sent(self, sent_info):
        result = []
        noun_ids = self.__get_all_noun_ids(sent_info)
        if len(noun_ids) == 0:
            return result

        token_graph = self.__build_token_graph(sent_info)

        for noun_id in noun_ids:
            noun_token = token_graph[noun_id]
            collocation = self.__extract_collocation_for_noun(noun_token)
            if len(collocation) > 0:
                result.append(collocation)

        return result

    def __extract_collocation_for_noun(self, noun_token):
        queue = self.__get_all_suitable_children(noun_token)
        if self.__is_head_suitable(noun_token):
            queue.append(noun_token.head)
        phrase_tokens = [noun_token]

        while queue:
            item = queue.pop(0)
            phrase_tokens.append(item)

            children = item.children

            for child in children:
                deprel = child.get_deprel()
                if deprel in self.IGNORE_NOUN_CHILD_DEPREL:
                    return []
                
                if child.get_deprel() == 'advmod':
                    queue.append(child)
                    
                if child.get_deprel() == 'conj':
                    queue.append(child)


        return self.__validate_noun_collocation(phrase_tokens)

    def __get_all_suitable_children(self, noun_token):
        queue = []

        children = noun_token.children

        for child in children:
            deprel = child.get_deprel()
            if deprel in self.SUPPORTED_NOUN_CHILD_DEPREL:
                queue.append(child)

        return queue

    def __is_head_suitable(self, noun_token):
        if noun_token.head is None:
            return False

        if noun_token.token_info['deprel'] not in self.SUPPORTED_NOUN_HEAD_DEPREL:
            return False

        if noun_token.head.token_info['upos'] not in self.SUPPORTED_HEAD_POS:
            return False

        return True

    def __build_token_graph(self, sent_info):
        token_node_dict = dict()
        for token_info in sent_info:
            token_id = int(token_info['id'])
            head_id = token_info['head']

            token_node_dict[token_id] = TokenNode(token_info)

        for token_id in token_node_dict:
            token_node = token_node_dict[token_id]

            head_id = token_node.get_head_id()

            if head_id > 0:
                token_node.head = token_node_dict[head_id]
                token_node_dict[head_id].children.append(token_node)

        return token_node_dict

    def __get_all_noun_ids(self, sent_info):
        ids = []
        for token_info in sent_info:
            if token_info['upos'] == 'NOUN':
                ids.append(int(token_info['id']))

        return ids

    def __validate_noun_collocation(self, collocation_tokens):
        result = []
        
        collocation_tokens.sort(key=lambda token: token.get_id())
        
        collocation_tokens = self.__find_max_len_non_sequence_words(collocation_tokens)
        
        if len(collocation_tokens) < 2:
            return []
        
        if self.__is_collocation_valid_by_pos(collocation_tokens):
            return [item.get_text().lower() for item in collocation_tokens]
        

        return []
    
    
    def __find_max_len_non_sequence_words(self, collocation_tokens):
        result = []
        if len(collocation_tokens) == 0:
            return result
        
        cur_seq = [collocation_tokens[0]]
        max_seq = cur_seq
        for i in range(1, len(collocation_tokens)):
            if collocation_tokens[i].get_id() - collocation_tokens[i - 1].get_id() > 1:
                if len(max_seq) < len(cur_seq):
                    max_seq = cur_seq
                cur_seq = []
            else:
                cur_seq.append(collocation_tokens[i])
                
        return max_seq
    
    
    def __is_collocation_valid_by_pos(self, collocation_tokens):
        noun_token, adj_token, verb_token = None, None, None
        for token in collocation_tokens:
            pos = token.get_pos()
            if pos == 'ADJ':
                adj_token = token
            elif pos == 'VERB':
                verb_token = token
            elif pos == 'NOUN':
                noun_token = token
        
        if noun_token is None:
            return False
        
        if verb_token is None and adj_token is None:
            return False
        
        
        if verb_token is None:
            if 'Case=Gen' in adj_token.get_feats():  # to avoid collocation like "гарячої води", but support "немає гарячої води":
                return False
        else:
            if noun_token.get_id() < verb_token.get_id(): #to avoid collocation like "колеги знайшли"
                return False
            
        return True

In [354]:
extractor = NounCollocationExtractor(nlp)

### Collocation example

In [314]:
text_example = "Не дуже привітний персонал, тому було некомфортно тут зупинятись."

In [315]:
extractor.extract(text_example)

[['не', 'дуже', 'привітний', 'персонал']]

In [317]:
doc = nlp(text_example)
doc.sentences[0].print_dependencies()

('Не', '2', 'advmod')
('дуже', '3', 'advmod')
('привітний', '4', 'amod')
('персонал', '0', 'root')
(',', '8', 'punct')
('тому', '8', 'advmod')
('було', '8', 'cop')
('некомфортно', '4', 'parataxis')
('тут', '10', 'advmod')
('зупинятись', '8', 'csubj')
('.', '4', 'punct')


#### short explanation

Як спрацював для даного речення алгоритм? <br>
Спочтаку знайшли іменник *персонал*. Далі до нього всих дітей (ті хто мають індекс head 4) і відповідний deprel (advmod або amod), тут це *привітний*. Для слова *привітний* також шукаються його діти з відповідним deprel, тут це *дуже*, потім для *дуже* знайшовся child *не*, для якого вже дітей немає і тому черга розгляду стала пустою. Отже маємо токени *не дуже привітний персонал*, які в сукупності проходять додаткову перевірки (так щою вони дійсно йшли по-порядку, іменник був після дієслова, якщо воно є та інші перевірки).

In [318]:
text_example = "Гарний готель, зручне розташування. Персонал ввічливий. Номера чисті, рушники сухі й чисті. В ванній кімнаті все охайно, є по 4 невеличких мила та 4 одноразових шампуня. А санвузлі є водонагрівач, горячої води було вдосталь. (Номер був на чотирьох осіб, мешкали двоє). Є одноразові тапочки. В номері було два невеличких лед-телевізора, мініхолодильник та кондиціонер. З вікна відкривається чудовий вид на Замкову гору. Поряд є шикарний пивний ресторан Кумпель. В готелі є камера схову, якщо треба висилятися а ще багато вільного часу - можна безкоштовно обставити речі та піти по своїх справах. Бал зняв за таргана, якого колеги знайшли в себе в номері. Підкреслю, що в нас такого не було."
text_example

'Гарний готель, зручне розташування. Персонал ввічливий. Номера чисті, рушники сухі й чисті. В ванній кімнаті все охайно, є по 4 невеличких мила та 4 одноразових шампуня. А санвузлі є водонагрівач, горячої води було вдосталь. (Номер був на чотирьох осіб, мешкали двоє). Є одноразові тапочки. В номері було два невеличких лед-телевізора, мініхолодильник та кондиціонер. З вікна відкривається чудовий вид на Замкову гору. Поряд є шикарний пивний ресторан Кумпель. В готелі є камера схову, якщо треба висилятися а ще багато вільного часу - можна безкоштовно обставити речі та піти по своїх справах. Бал зняв за таргана, якого колеги знайшли в себе в номері. Підкреслю, що в нас такого не було.'

In [319]:
extractor.extract(text_example)

[['гарний', 'готель'],
 ['зручне', 'розташування'],
 ['персонал', 'ввічливий'],
 ['рушники', 'сухі'],
 ['ванній', 'кімнаті'],
 ['є', 'водонагрівач'],
 ['є', 'одноразові', 'тапочки'],
 ['відкривається', 'чудовий', 'вид'],
 ['замкову', 'гору'],
 ['поряд', 'є', 'шикарний', 'пивний', 'ресторан', 'кумпель'],
 ['є', 'камера', 'схову']]

In [320]:
text_example = "Маленький затишний готель у гарному й тихому районі. Номер досить просторий. Ліжко зручне. Є фен та халат, шафа, настільна лампа, стіл та 2 стільця, прикроватні тумбочки та зручно розташовані розетки, мінібар. Сніданок входить до вартості. Вибір на сніданок не великий, але все смачно. Окрім маленького шведського столу можна замовити млинці, куряче філе, яєшню чи ще щось (теж входить в ціну). Недалеко від готелю, дерев'яні сходи вгору до Пейзажної алеї та Музеї історії України."
text_example

"Маленький затишний готель у гарному й тихому районі. Номер досить просторий. Ліжко зручне. Є фен та халат, шафа, настільна лампа, стіл та 2 стільця, прикроватні тумбочки та зручно розташовані розетки, мінібар. Сніданок входить до вартості. Вибір на сніданок не великий, але все смачно. Окрім маленького шведського столу можна замовити млинці, куряче філе, яєшню чи ще щось (теж входить в ціну). Недалеко від готелю, дерев'яні сходи вгору до Пейзажної алеї та Музеї історії України."

In [321]:
extractor.extract(text_example)

[['маленький', 'затишний', 'готель'],
 ['номер', 'досить', 'просторий'],
 ['ліжко', 'зручне'],
 ['є', 'фен'],
 ['настільна', 'лампа'],
 ['прикроватні', 'тумбочки'],
 ['зручно', 'розташовані', 'розетки'],
 ["дерев'яні", 'сходи', 'вгору']]

TODO: think how to avoid collocation **настільна лампа**, **прикроватні тумбочки**, **деревяні сходи вгору**

In [325]:
doc = nlp(text_example)
doc.sentences[3].print_dependencies()

('Є', '0', 'root')
('фен', '1', 'nsubj')
('та', '4', 'cc')
('халат', '2', 'conj')
(',', '6', 'punct')
('шафа', '2', 'conj')
(',', '9', 'punct')
('настільна', '9', 'amod')
('лампа', '2', 'conj')
(',', '11', 'punct')
('стіл', '2', 'conj')
('та', '14', 'cc')
('2', '14', 'nummod:gov')
('стільця', '2', 'conj')
(',', '17', 'punct')
('прикроватні', '17', 'amod')
('тумбочки', '2', 'conj')
('та', '20', 'cc')
('зручно', '20', 'advmod')
('розташовані', '1', 'conj')
('розетки', '20', 'nsubj')
(',', '23', 'punct')
('мінібар', '21', 'appos')
('.', '1', 'punct')


In [331]:
doc.sentences[7].print_dependencies()

('Недалеко', '0', 'root')
('від', '3', 'case')
('готелю', '1', 'obl')
(',', '6', 'punct')
("дерев'яні", '6', 'amod')
('сходи', '1', 'advcl')
('вгору', '6', 'advmod')
('до', '10', 'case')
('Пейзажної', '10', 'amod')
('алеї', '7', 'obl')
('та', '12', 'cc')
('Музеї', '10', 'conj')
('історії', '12', 'nmod')
('України', '13', 'nmod')
('.', '1', 'punct')


In [326]:
doc.sentences[3]

[
  {
    "id": "1",
    "text": "Є",
    "lemma": "бути",
    "upos": "VERB",
    "xpos": "Vapip3s",
    "feats": "Aspect=Imp|Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin",
    "head": 0,
    "deprel": "root",
    "misc": "start_char=91|end_char=92"
  },
  {
    "id": "2",
    "text": "фен",
    "lemma": "фен",
    "upos": "NOUN",
    "xpos": "Ncmsnn",
    "feats": "Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing",
    "head": 1,
    "deprel": "nsubj",
    "misc": "start_char=93|end_char=96"
  },
  {
    "id": "3",
    "text": "та",
    "lemma": "та",
    "upos": "CCONJ",
    "xpos": "Ccs",
    "head": 4,
    "deprel": "cc",
    "misc": "start_char=97|end_char=99"
  },
  {
    "id": "4",
    "text": "халат",
    "lemma": "халат",
    "upos": "NOUN",
    "xpos": "Ncmsnn",
    "feats": "Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing",
    "head": 2,
    "deprel": "conj",
    "misc": "start_char=100|end_char=105"
  },
  {
    "id": "5",
    "text": ",",
    "lemma": ",",
    "upos"

In [338]:
text_example = "Дуже затишний, надзвичайно чистий готель. Хоча ми приїхали дуже пізно, нас чекали і привітно обслужили. Вранці ми виїзджали дуже рано в аеропорт, тому замість сніданку нам запропонували сендвічі в дорогу. А також забезпечили трансфером. Дякуємо за хороший сервіс!."
text_example

'Дуже затишний, надзвичайно чистий готель. Хоча ми приїхали дуже пізно, нас чекали і привітно обслужили. Вранці ми виїзджали дуже рано в аеропорт, тому замість сніданку нам запропонували сендвічі в дорогу. А також забезпечили трансфером. Дякуємо за хороший сервіс!.'

In [339]:
extractor.extract(text_example)

[['хороший', 'сервіс']]

In [340]:
doc = nlp(text_example)
doc.sentences[0].print_dependencies()

('Дуже', '2', 'advmod')
('затишний', '6', 'amod')
(',', '5', 'punct')
('надзвичайно', '5', 'advmod')
('чистий', '2', 'conj')
('готель', '0', 'root')
('.', '6', 'punct')


Як бачимо вище колокація *читий готель* не врахувалась, із-за того що немає звязку між читий і готель, а готель вказує на затишний, що в свою чергу містить child чистий. Варто подумати як це акууратно теж врахувати.

In [335]:
text_example = "Дуже хороший готель! Привітні люди, почуваєш себе як в дома 😍🤗."

In [336]:
extractor.extract(text_example)

[['дуже', 'хороший', 'готель'], ['привітні', 'люди']]

In [341]:
text_example = "Номер був розрахований на трьох, та ліжко (розкладний диван) для третього чоловіка був просто непридатний для того, щоб на ньому спати, це був просто жах! Сніданок - це щось смішне.."

In [342]:
extractor.extract(text_example)

[['розкладний', 'диван']]

In [343]:
text_example = "Чудовий готель! Дуже привітний персонал, красивий та чистий номер. Надзвичайно смачний сніданок та можливість на місці замовити не менш смачні перекуси чи чай/каву. Красивий та затишний задній дворик-сад де чудово можна посидіти та помилуватись заходом сонця. Лише позитивні емоції від готелю."

In [344]:
extractor.extract(text_example)

[['чудовий', 'готель'],
 ['дуже', 'привітний', 'персонал'],
 ['надзвичайно', 'смачний', 'сніданок'],
 ['не', 'менш', 'смачні', 'перекуси'],
 ['позитивні', 'емоції']]

In [355]:
extractor.extract("Гарний готель, зручне розташування розеток.")

[['гарний', 'готель'], ['зручне', 'розташування', 'розеток']]

In [356]:
text1 = "В ванній кімнаті все охайно, є по 4 невеличких мила та 4 одноразових шампуня."

In [357]:
extractor.extract(text1)

[['ванній', 'кімнаті']]

In [358]:
extractor.extract("Немає одноразових тапочок.")

[]

TODO: this collocation should be considered

In [359]:
extractor.extract("Одноразові тапочки відсутні.")

[['одноразові', 'тапочки', 'відсутні']]

In [360]:
extractor.extract("Не дуже привітний персонал, тому було некомфортно тут зупинятись.")

[['не', 'дуже', 'привітний', 'персонал']]

### Collocations in positive reviews text

In [364]:
pos_texts = train_df['pos_text'].values[:1000]

In [365]:
pos_collocations = []
for text in pos_texts:
    collocations = extractor.extract(text)
    if len(collocations) > 0:
        pos_collocations.append(collocations)

In [366]:
pos_collocations

[[['оформлення', 'кімнати', 'хороше'], ['чудовий', 'варіант']],
 [['якісному', 'сервісу'],
  ['надзвичайно', "люб'язний", 'персонал'],
  ['є', 'територія']],
 [['свіжий', 'ремонт']],
 [['не', 'перший', 'раз']],
 [['достатня', 'кількість'], ['мікрохвильова', 'піч'], ['повному', 'обсязі']],
 [['готель', 'дуже', 'класний']],
 [['зручне', 'ліжко'], ['відповідає', 'фото']],
 [['зручне', 'розташування'], ['привітний', 'персонал']],
 [['чудовим', 'ліжком'], ['привітний', 'персонал']],
 [['дуже', 'привітний', 'персонал'],
  ['ідеальна', 'чистота'],
  ['доступні', 'ціни']],
 [['не', 'погана', 'шумоізоляція']],
 [['безкоштовна', 'парковка'], ['дуже', 'великий', 'номер']],
 [['доволі', 'просторий', 'номер']],
 [['банківську', 'картку'], ['завжди', 'є', 'місця']],
 [['оптимальне', 'співвідношення'], ['гарні', 'умови'], ['невисоку', 'плату']],
 [['зручний', 'комфортний', 'номер'], ['необхідними', 'зручностями']],
 [['сімейний', 'номер'], ['номер', 'чистий'], ['дитяче', 'ліжечко']],
 [['розумне', 'с

TODO: проаналізувати чому потрапляє шум. Наприклад така колокація як **кращу ціну** складно відрізнити від типу колокації роду **гарне розташування**. 

### Collocations in negative reviews text

In [367]:
neg_texts = train_df['neg_text'].values[:1000]

In [369]:
neg_collocations = []
for text in neg_texts:
    try:
        collocations = extractor.extract(text)
        if len(collocations) > 0:
            neg_collocations.append(collocations)
    except:
        pass

In [370]:
neg_collocations

[[['байдуже', 'умови']],
 [['власним', 'транспортом']],
 [['особливо', 'не', 'сподобалась', 'басовито', 'музика']],
 [['8', 'поверсі'], ['цілу', 'ніч']],
 [['постільній', 'білизні', 'волосся'], ['постільну', 'білизну']],
 [['зламані', 'гачки'], ['розбитий', 'стаканчик']],
 [['3', 'корпусі'], ['будуть', 'номери'], ['розкішних', 'умовах']],
 [['маленькою', 'дитиною']],
 [['холодна', 'кахельні', 'підлога'],
  ['не', 'працював', 'кондиціонер'],
  ['ванній', 'кімнаті']],
 [['3-му', 'корпусі'], ['номери', 'більш', 'охайні']],
 [['неприємний', 'день']],
 [['персонал', 'не', 'дуже', 'привітний'],
  ['нормально', 'не', 'стікає', 'вода'],
  ['підлога', 'постійно', 'мокра']],
 [['5', 'корпусі'], ['тимчасові', 'незручності']],
 [['не', 'уважний', 'персонал']],
 [['витікала', 'вода'], ['одноразовими', 'шапочками'], ['гаряча', 'вода']],
 [['дуже', 'погана', 'шумоізоляція'],
  ['не', 'працює', 'пристрій'],
  ['там', 'ще', 'ведуться', 'ремонти', 'тому']],
 [['був', 'слабкий', 'неприємний', 'запах']],


Як бачимо для з негативних коментрів більше шуму в колокаціях.